# Restaurant locations in Los Angeles

('Trust this notebook' to see the folium mapping)

In [2]:
import pandas as pd
import numpy as np
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

from collections import Counter 
from geopy.geocoders import Nominatim
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
CLIENT_ID='HM3AAQI50VK0VMIFVRGE1WATLARDR3EHLEYY1RAVJFN3PRKE'
CLIENT_SECRET='ENWI1Y4UVIVQ13MEKRL5FKYYTHNE5JAXRF2JASLIOG05JRX4'
ACCESS_TOKEN='2I3X5M1HPK3OJEM14M3VXUOEKFQ0VSY2BUYIC2N0D5L4THGZ'

In [3]:
# functions
def getList(dict): 
    return dict.keys() 

def unique_list(input_list):
    output_list = []
    for word in input_list:
        if word not in output_list:
            output_list.append(word)
    print(output_list)
    return output_list

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [4]:
address = 'Los Angeles, CA'
geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude)) 

The geograpical coordinate of Los Angeles are 34.0536909, -118.242766.


In [5]:
# future update
# def f(venue):
#     search_query = venue
#     radius = 2000000
#     url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
#     return url
#w1=interact(f, venue=['Restaurant','Pub'])
#display(w1)

search_query = 'Restaurant'
radius = 2000000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)


In [6]:
results = requests.get(url).json()
venues = results['response']['venues']
df = json_normalize(venues)
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
df_filtered = df.loc[:, filtered_columns]
# filter the category for each row
df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]


Category_list=unique_list(df_filtered.categories)

['French Restaurant', 'American Restaurant', 'New American Restaurant', 'Mexican Restaurant', 'Chinese Restaurant', 'Seafood Restaurant', 'Italian Restaurant', 'Salad Place', 'Asian Restaurant', 'Latin American Restaurant', 'Sushi Restaurant', 'Restaurant', 'Mediterranean Restaurant', 'Indian Restaurant', 'Kitchen Supply Store', 'Japanese Restaurant', 'Spanish Restaurant', 'Construction & Landscaping', 'Food', 'Thai Restaurant', None, 'Korean Restaurant', 'Taco Place', 'Breakfast Spot', 'Dim Sum Restaurant', 'Cajun / Creole Restaurant']


In [7]:

# count each type of restaurant and show most frequent types in Los Angeles
Full_category_list=df_filtered.categories
Counter_list=Counter(Full_category_list) # equals to list(set(words))
Counter_name=list(Counter_list)
Counter_count=list(Counter_list.values())
IND_Counter_bool=np.array(Counter_count)>1 # flag to remove below 1
IND_Counter= np.where(IND_Counter_bool)[0]

#for i in range(len(IND_Counter)): 
#   print(Counter_name[IND_Counter[i]],": ", str(Counter_count[IND_Counter[i]]))

In [8]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Ecco

Color_list=['red', 'blue', 'green', 'purple', 'orange', 'darkred',
'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

# add all markers without categories
for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='skyblue',
        fill_color='skyblue',
        fill_opacity=0.6
        ).add_to(venues_map)
Name_list=[]
for i in range(len(IND_Counter)):
    Name_list.append(Counter_name[IND_Counter[i]])
    print(Counter_name[IND_Counter[i]])
    for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.categories):  
        if label==Counter_name[IND_Counter[i]]:           
            folium.CircleMarker(
                [lat, lng],
                radius=5,
                popup=label,
                fill=True,
                color=Color_list[i],
                fill_color=Color_list[i],
                fill_opacity=0.6
                ).add_to(venues_map)
Name_list

lgd_txt = '<span style="color: {col};">{txt}</span>'

for idx, Name in enumerate(Name_list):  # color choice is limited
    Legend = folium.FeatureGroup(name= lgd_txt.format( txt= Name, col= Color_list[idx]))
    venues_map.add_child(Legend)

folium.map.LayerControl('topleft', collapsed= False).add_to(venues_map)            
 
venues_map

French Restaurant
American Restaurant
New American Restaurant
Mexican Restaurant
Chinese Restaurant
Italian Restaurant
Asian Restaurant
Latin American Restaurant
Restaurant
Mediterranean Restaurant
Japanese Restaurant
None


In [9]:
for i in range(len(IND_Counter)): 
   print(Counter_name[IND_Counter[i]],": ", str(Counter_count[IND_Counter[i]]))

French Restaurant :  3
American Restaurant :  2
New American Restaurant :  2
Mexican Restaurant :  6
Chinese Restaurant :  5
Italian Restaurant :  4
Asian Restaurant :  2
Latin American Restaurant :  2
Restaurant :  2
Mediterranean Restaurant :  2
Japanese Restaurant :  4
None :  2
